# Pipeline tiền xử lý dữ liệu
Chuẩn bị dữ liệu ảnh cho bài toán phân loại bệnh lá lúa.

In [8]:
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25

DATA_DIR = Path("../data")
TRAIN_IMG_DIR = DATA_DIR / "train_images"
TRAIN_CSV_PATH = DATA_DIR / "train.csv"

LABEL_MAP = {
    "bacterial_leaf_blight": "Bạc lá (Vi khuẩn)",
    "bacterial_leaf_streak": "Đốm sọc (Vi khuẩn)",
    "bacterial_panicle_blight": "Lép hạt (Vi khuẩn)",
    "blast": "Đạo ôn",
    "brown_spot": "Đốm nâu",
    "dead_heart": "Sâu đục thân",
    "downy_mildew": "Sương mai",
    "hispa": "Bọ gai",
    "normal": "Bình thường",
    "tungro": "Tungro (Virus)",
}


In [9]:
# # 1. KHỞI TẠO IMAGE DATA GENERATOR

# datagen_train = ImageDataGenerator(
#     rescale=1.0 / 255,
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode="nearest",
#     validation_split=0.2,
#     )

# datagen_valid = ImageDataGenerator(
#     rescale=1.0 / 255,
#     validation_split=0.2,
#     )

# print("Đã khởi tạo ImageDataGenerator (train: augment + rescale, valid: rescale).")


In [10]:
# 1. KHỞI TẠO IMAGE DATA GENERATOR 

datagen_train = ImageDataGenerator(
    rescale=1.0 / 255,

    # rotation_range=20,        # baseline
    rotation_range=25,          # ↑

    # width_shift_range=0.1,    # baseline
    # height_shift_range=0.1,
    width_shift_range=0.15,     
    height_shift_range=0.15,

    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,

    brightness_range=[0.7, 1.3],  # 

    fill_mode="nearest",
    validation_split=0.2,
)

datagen_valid = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
)

print("Đã khởi tạo ImageDataGenerator (train: augment + rescale, valid: rescale).")


Đã khởi tạo ImageDataGenerator (train: augment + rescale, valid: rescale).


In [13]:
# 2. TẠO GENERATOR TRAIN/VALID

train_generator = datagen_train.flow_from_directory(
    str(TRAIN_IMG_DIR),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=SEED,
    subset="training",
)

valid_generator = datagen_valid.flow_from_directory(
    str(TRAIN_IMG_DIR),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=SEED,
    subset="validation",
)

# Lấy thứ tự lớp theo index của Keras
class_indices = train_generator.class_indices
class_names_en = [cls for cls, idx in sorted(class_indices.items(), key=lambda x: x[1])]
class_names_vn = [LABEL_MAP.get(cls, cls) for cls in class_names_en]

for idx, (en, vi) in enumerate(zip(class_names_en, class_names_vn)):
    print(f"lớp {idx}: {en} -> {vi}")

print("\nDanh sách: ", class_names_vn)
print("Tổng số lớp:", len(class_names_vn))


Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
lớp 0: bacterial_leaf_blight -> Bạc lá (Vi khuẩn)
lớp 1: bacterial_leaf_streak -> Đốm sọc (Vi khuẩn)
lớp 2: bacterial_panicle_blight -> Lép hạt (Vi khuẩn)
lớp 3: blast -> Đạo ôn
lớp 4: brown_spot -> Đốm nâu
lớp 5: dead_heart -> Sâu đục thân
lớp 6: downy_mildew -> Sương mai
lớp 7: hispa -> Bọ gai
lớp 8: normal -> Bình thường
lớp 9: tungro -> Tungro (Virus)

Danh sách:  ['Bạc lá (Vi khuẩn)', 'Đốm sọc (Vi khuẩn)', 'Lép hạt (Vi khuẩn)', 'Đạo ôn', 'Đốm nâu', 'Sâu đục thân', 'Sương mai', 'Bọ gai', 'Bình thường', 'Tungro (Virus)']
Tổng số lớp: 10
